# 잔액정보 EDA

In [1]:
import sys
sys.path.append('../utils')

## 데이터 로드

In [2]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
from preprocessing import (
    DateElapsedTransformer, NumericFeaturePreprocessor, ObjectFeaturePreprocessor, NumericTypeOptimizer,
)

pre_optimizer = NumericTypeOptimizer(mode='pre')
date_transformer = DateElapsedTransformer()
numeric_transformer = NumericFeaturePreprocessor()
object_transformer = ObjectFeaturePreprocessor()
post_optimizer = NumericTypeOptimizer(mode='post')

preprocessing_pipeline = Pipeline([
    ('pre_optimizer', pre_optimizer),
    ('object', object_transformer),
    ('date', date_transformer),
    ('numeric', numeric_transformer),
    ('post_optimizer', post_optimizer),
])

# Set display configuration to visualize the pipeline
set_config(display='diagram')

# Display the pipeline
preprocessing_pipeline

Pipeline(steps=[('pre_optimizer', NumericTypeOptimizer(mode='pre')),
                ('object',
                 ObjectFeaturePreprocessor(exclude_columns=['ID', 'Segment'])),
                ('date',
                 DateElapsedTransformer(base_date=Timestamp('2019-01-01 00:00:00'),
                                        exclude_columns=['ID', 'Segment',
                                                         '기준년월'])),
                ('numeric',
                 NumericFeaturePreprocessor(exclude_columns=['기준년월'])),
                ('post_optimizer', NumericTypeOptimizer())])

In [3]:
from sklearn.preprocessing import LabelEncoder
from data_loader import (
    load_data,
    load_segment
)

balance_df, balance_test_df = load_data("5.잔액정보")

X = balance_df.drop(columns=['ID', '기준년월'])
X = preprocessing_pipeline.fit_transform(X)
X_test = preprocessing_pipeline.transform(balance_test_df.drop(columns=['ID', '기준년월']))

y = load_segment()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

✅ File: ../../dataset/train/5.잔액정보\201807_train_잔액정보.parquet Completed!
✅ File: ../../dataset/train/5.잔액정보\201808_train_잔액정보.parquet Completed!
✅ File: ../../dataset/train/5.잔액정보\201809_train_잔액정보.parquet Completed!
✅ File: ../../dataset/train/5.잔액정보\201810_train_잔액정보.parquet Completed!
✅ File: ../../dataset/train/5.잔액정보\201811_train_잔액정보.parquet Completed!
✅ File: ../../dataset/train/5.잔액정보\201812_train_잔액정보.parquet Completed!
🔹 Shape : (2400000, 82)

✅ File: ../../dataset/test/5.잔액정보\201807_test_잔액정보.parquet Completed!
✅ File: ../../dataset/test/5.잔액정보\201808_test_잔액정보.parquet Completed!
✅ File: ../../dataset/test/5.잔액정보\201809_test_잔액정보.parquet Completed!
✅ File: ../../dataset/test/5.잔액정보\201810_test_잔액정보.parquet Completed!
✅ File: ../../dataset/test/5.잔액정보\201811_test_잔액정보.parquet Completed!
✅ File: ../../dataset/test/5.잔액정보\201812_test_잔액정보.parquet Completed!
🔹 Shape : (600000, 82)
Numeric Type Optimizer Transforming...
🧠 [mode=pre] 메모리 최적화: 1464.84 MB → 565.34 MB (61.4% 감소)
Objec

## Feature Selection

In [4]:
from feature import FeatureSelector

feature_engineering_pipeline = Pipeline([
    ('feature_selector', FeatureSelector()),
    ('optimize', post_optimizer),
])

X = feature_engineering_pipeline.fit_transform(X, y_encoded)
X_test = feature_engineering_pipeline.transform(X_test)

🎯 Fitting RandomForest for Feature Selection...
✅ Selected 17 features out of 73
📊 Top Selected Features by Importance:
1) 	잔액_일시불_B0M (0.0458)
2) 	월중평잔_일시불_B0M (0.1139)
3) 	잔액_현금서비스_B1M (0.0152)
4) 	잔액_현금서비스_B2M (0.0145)
5) 	잔액_일시불_B1M (0.0337)
6) 	잔액_일시불_B2M (0.0309)
7) 	월중평잔 (0.0430)
8) 	월중평잔_일시불 (0.1049)
9) 	월중평잔_CA (0.0168)
10) 	평잔_3M (0.0784)
11) 	평잔_일시불_3M (0.0456)
12) 	평잔_CA_3M (0.0188)
13) 	평잔_6M (0.0924)
14) 	평잔_일시불_6M (0.0766)
15) 	평잔_일시불_해외_6M (0.0212)
16) 	평잔_할부_6M (0.0270)
17) 	평잔_CA_6M (0.0295)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 311.28 MB → 151.06 MB (51.5% 감소)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 77.82 MB → 37.77 MB (51.5% 감소)


In [5]:
import pandas as pd

train_df = pd.concat([balance_df[['ID', '기준년월']], X], axis=1)
test_df = pd.concat([balance_test_df[['ID', '기준년월']], X_test], axis=1)

train_df.to_parquet('../../dataset/train/balance.parquet', index=False)
test_df.to_parquet('../../dataset/test/balance.parquet', index=False)